In [1]:
import torch
from torch_geometric.data import HeteroData

# Initialize the node features for each type of node
# For this example, let's say each node A has 5 features, and nodes B1 and B2 do not have features
num_nodes_A = 1000
num_nodes_B1 = 500
num_nodes_B2 = 500

x_A = torch.randn((num_nodes_A, 5))  # 5 features for each node A
x_B1 = torch.zeros((num_nodes_B1, 0))  # no features for nodes B1
x_B2 = torch.zeros((num_nodes_B2, 0))  # no features for nodes B2

# Initialize the edges for each type of edge
# For this example, let's say we have 2000 edges between nodes A and B1, 
# and 3000 edges between nodes B1 and B2
num_edges_A_B1 = 2000
num_edges_B1_B2 = 3000

edge_index_A_B1 = torch.randint(0, num_nodes_A, (2, num_edges_A_B1))
edge_index_B1_B2 = torch.randint(0, num_nodes_B1, (2, num_edges_B1_B2))

# Initialize the labels for each type of edge
# For this example, let's say we're doing binary classification, so labels are 0 or 1
y_A_B1 = torch.randint(0, 2, (num_edges_A_B1,))
y_B1_B2 = torch.randint(0, 2, (num_edges_B1_B2,))

# Create a HeteroData object
data = HeteroData()

# Add node data
data['A'].x = x_A
data['B1'].x = x_B1
data['B2'].x = x_B2

# Add edge data
data['A', 'connects', 'B1'].edge_index = edge_index_A_B1
data['A', 'connects', 'B1'].y = y_A_B1

data['B1', 'connects', 'B2'].edge_index = edge_index_B1_B2
data['B1', 'connects', 'B2'].y = y_B1_B2

In [8]:
x_B1

tensor([], size=(500, 0))

In [7]:
data['B1', 'connects', 'B2'].edge_index

tensor([[173, 436, 416,  ...,  48,  74, 306],
        [289, 254, 185,  ...,  68, 143, 399]])

In [9]:
data['A', 'connects', 'B1'].y

tensor([0, 1, 1,  ..., 0, 1, 1])

In [10]:
torch.ones(10)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])